In [1]:
import pandas as pd
import datetime
from datetime import timedelta
import time
from time import mktime
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
from plotly.graph_objs.scatter.marker import Line
from plotly.subplots import make_subplots
import xlrd
import openpyxl
import numpy as np
import geopandas
import json
import math
import altair as alt

In [151]:
soco = pd.read_csv('sonoma_county_inout.csv', parse_dates=['Date Of Birth','Intake Date','Outcome Date'])
# nyc311 = pd.read_csv('nyc_311.csv', parse_dates=['Created Date','Closed Date','Resolution Action Updated Date'])
# bloom = pd.read_csv('bloomington_animal_shelter.csv', parse_dates=['intakedate','movementdate','returndate'])
# aac_in = pd.read_csv('aac_intakes.csv', parse_dates=['datetime','datetime2'])
# aac_out = pd.read_csv('aac_outcomes.csv', parse_dates=['date_of_birth','datetime','monthyear'])
# aac_inout = pd.read_csv('aac_intakes_outcomes.csv', parse_dates=['date_of_birth','outcome_datetime',
#                                                                  'intake_datetime'])

In [138]:
def format_data(d, title_columns, col_order=[]):
    for t in title_columns:
        d[t] = d[t].str.title()
    
    d.columns = [x.lower().replace(' ','_') for x in d.columns]
    
    if col_order != []:
        d = d[col_order]
      
    return d

soco = format_data(soco, ['Name','Type','Breed','Color','Size','Intake Type','Intake Subtype',
                          'Outcome Type','Outcome Subtype','Intake Condition','Outcome Condition',
                          'Intake Jurisdiction','Outcome Jurisdiction'],
                   col_order = ['name','type','breed','color','sex','size','date_of_birth','impound_number',
                                'kennel_number','animal_id','intake_date','outcome_date','days_in_shelter',
                                'intake_type','intake_subtype','outcome_type','outcome_subtype',
                                'intake_condition','outcome_condition','intake_jurisdiction','outcome_jurisdiction',
                                'outcome_zip_code','location'])

nyc311 = format_data(nyc311, 
                     ['Incident Address','Street Name','Cross Street 1','Cross Street 2',
                      'Intersection Street 1','Intersection Street 2','Address Type',
                      'City','Landmark'],
                     col_order = ['unique_key','created_date','agency_name','complaint_type',
                                 'descriptor','location_type','incident_zip','borough','latitude',
                                 'longitude','location'])

In [152]:
soco.iloc[:,:20]

,Name,Type,Breed,Color,Sex,Size,Date Of Birth,Impound Number,Kennel Number,Animal ID,Intake Date,Outcome Date,Days in Shelter,Intake Type,Intake Subtype,Outcome Type,Outcome Subtype,Intake Condition,Outcome Condition,Intake Jurisdiction
0,REMY,DOG,BEAGLE,BLACK/WHITE,Female,MED,2014-04-11,K21-039564,DS69,A406645,2021-09-08,2021-09-11,3,STRAY,PHONE,RETURN TO OWNER,OVER THE COUNTER_WEB,UNKNOWN,PENDING,SANTA ROSA
1,*RONDE,DOG,CHIHUAHUA SH,TAN,Neutered,SMALL,2020-07-24,K21-039133,DA30,A406003,2021-07-24,2021-09-11,49,STRAY,FIELD,ADOPTION,SCAS WEB,HEALTHY,HEALTHY,SANTA ROSA
2,*SNOOP,CAT,DOMESTIC SH,GRAY,Neutered,SMALL,2019-08-04,K21-039232,CA02,A406151,2021-08-04,2021-09-11,38,STRAY,FIELD,ADOPTION,SCAS WEB,HEALTHY,HEALTHY,SANTA ROSA
3,FIFI,CAT,SIAMESE,SEAL PT,Spayed,KITTN,2020-10-27,K21-037796,CA03,A402286,2021-01-30,2021-02-05,6,OWNER SURRENDER,OVER THE COUNTER,ADOPTION,SCAS WEB,UNKNOWN,PENDING,SANTA ROSA
4,CINNAMON,DOG,PIT BULL,BR BRINDLE,Female,MED,2016-09-27,K21-039600,DS66,A359646,2021-09-10,2021-09-11,1,STRAY,OVER THE COUNTER,RETURN TO OWNER,OVER THE COUNTER_WEB,UNKNOWN,PENDING,SANTA ROSA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22249,NaN,CAT,DOMESTIC SH,GRAY/WHITE,Female,SMALL,NaT,K19-031738,SMM,A385051,2019-03-21,2019-03-21,0,STRAY,OVER THE COUNTER,EUTHANIZE,ILL SEVERE,UNKNOWN,UNTREATABLE,SANTA ROSA
22250,NaN,CAT,DOMESTIC SH,GRAY,Unknown,KITTN,NaT,K18-027667,RECEIVING,A370465,2018-03-27,2018-03-27,0,STRAY,COMM CAT,TRANSFER,FORGOTTEN,UNKNOWN,HEALTHY,COUNTY
22251,NaN,DOG,CHIHUAHUA LH/MIX,BLACK/WHITE,Male,TOY,2017-02-03,K18-027158,DA16,A367906,2018-02-03,2018-02-16,13,STRAY,OVER THE COUNTER,TRANSFER,RP SHELTER,HEALTHY,HEALTHY,COUNTY
22252,ANDY,DOG,CHIHUAHUA LH/MIX,BROWN/WHITE,Neutered,TOY,2015-04-24,K17-022896,LOBBY,A349854,2017-03-23,2017-04-08,16,STRAY,OVER THE COUNTER,ADOPTION,INTERNET,TREATABLE/REHAB,TREATABLE/REHAB,SANTA ROSA


In [153]:
soco.select_dtypes(include=['object']).apply(lambda x: x.str.replace('*','', regex=True).str.title())

,Name,Type,Breed,Color,Sex,Size,Impound Number,Kennel Number,Animal ID,Days in Shelter,Intake Type,Intake Subtype,Outcome Type,Outcome Subtype,Intake Condition,Outcome Condition,Intake Jurisdiction,Outcome Jurisdiction,Location
0,Remy,Dog,Beagle,Black/White,Female,Med,K21-039564,Ds69,A406645,3,Stray,Phone,Return To Owner,Over The Counter_Web,Unknown,Pending,Santa Rosa,Santa Rosa,"95407(38.403734, -122.736775)"
1,Ronde,Dog,Chihuahua Sh,Tan,Neutered,Small,K21-039133,Da30,A406003,49,Stray,Field,Adoption,Scas Web,Healthy,Healthy,Santa Rosa,Petaluma,"94954(38.245316, -122.59871)"
2,Snoop,Cat,Domestic Sh,Gray,Neutered,Small,K21-039232,Ca02,A406151,38,Stray,Field,Adoption,Scas Web,Healthy,Healthy,Santa Rosa,Santa Rosa,"95405(38.439152, -122.672541)"
3,Fifi,Cat,Siamese,Seal Pt,Spayed,Kittn,K21-037796,Ca03,A402286,6,Owner Surrender,Over The Counter,Adoption,Scas Web,Unknown,Pending,Santa Rosa,Santa Rosa,"95407(38.403734, -122.736775)"
4,Cinnamon,Dog,Pit Bull,Br Brindle,Female,Med,K21-039600,Ds66,A359646,1,Stray,Over The Counter,Return To Owner,Over The Counter_Web,Unknown,Pending,Santa Rosa,Santa Rosa,"95407(38.403734, -122.736775)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22249,NaN,Cat,Domestic Sh,Gray/White,Female,Small,K19-031738,Smm,A385051,0,Stray,Over The Counter,Euthanize,Ill Severe,Unknown,Untreatable,Santa Rosa,NaN,NaN
22250,NaN,Cat,Domestic Sh,Gray,Unknown,Kittn,K18-027667,Receiving,A370465,0,Stray,Comm Cat,Transfer,Forgotten,Unknown,Healthy,County,Santa Rosa,"95403(38.486997, -122.749134)"
22251,NaN,Dog,Chihuahua Lh/Mix,Black/White,Male,Toy,K18-027158,Da16,A367906,13,Stray,Over The Counter,Transfer,Rp Shelter,Healthy,Healthy,County,Rohnert Park,"94928(38.346718, -122.695947)"
22252,Andy,Dog,Chihuahua Lh/Mix,Brown/White,Neutered,Toy,K17-022896,Lobby,A349854,16,Stray,Over The Counter,Adoption,Internet,Treatable/Rehab,Treatable/Rehab,Santa Rosa,Out Of County,"94109(37.792737, -122.420971)"


In [135]:
soco['count'].value_counts(dropna=False)

1    22199
0       55
Name: count, dtype: int64

In [132]:
soco.outcome_jurisdiction.value_counts(dropna=False)

Santa Rosa       7068
County           6313
NaN              3127
Out Of County    2546
*Windsor         1300
*Rohnert Park     691
*Petaluma         582
*Healdsburg       204
*Cloverdale       167
*Sebastopol       114
*Cotati            82
*Sonoma            53
Unknown             7
Name: outcome_jurisdiction, dtype: int64

In [136]:
soco[soco['count']==0]

,name,type,breed,color,sex,size,date_of_birth,impound_number,kennel_number,animal_id,...,intake_subtype,outcome_type,outcome_subtype,intake_condition,outcome_condition,intake_jurisdiction,outcome_jurisdiction,outcome_zip_code,location,count
62,*Persephone,Dog,Siberian Husky,Black/White,Spayed,Small,2020-04-03,K21-038184,DA37,A403451,...,Phone,Adoption,Scas Web,Healthy,Healthy,Santa Rosa,County,95425.0,"95425(38.790859, -123.010876)",0
94,Luna,Dog,Pit Bull/Mix,Black/White,Spayed,Med,2015-10-02,K20-037004,DA16,A341813,...,Field,Adoption,Scas Web,Unknown,Pending,County,County,95472.0,"95472(38.394388, -122.850998)",0
137,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038465,RECEIVING,A404464,...,Over The Counter,Transfer,Hssc,Unknown,Treatable/Rehab,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
138,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038459,RECEIVING,A404458,...,Over The Counter,Transfer,Hssc,Unknown,Healthy,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
139,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038462,RECEIVING,A404461,...,Over The Counter,Transfer,Hssc,Unknown,Treatable/Rehab,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
140,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038463,RECEIVING,A404462,...,Over The Counter,Transfer,Hssc,Unknown,Treatable/Rehab,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
143,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038456,RECEIVING,A404454,...,Over The Counter,Transfer,Hssc,Unknown,Healthy,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
145,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038464,RECEIVING,A404463,...,Over The Counter,Transfer,Hssc,Unknown,Treatable/Rehab,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
150,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038457,RECEIVING,A404456,...,Over The Counter,Transfer,Hssc,Unknown,Healthy,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
153,NaN,Cat,Domestic Sh,Agouti,Unknown,Kittn,NaT,K21-038466,RECEIVING,A404465,...,Over The Counter,Transfer,Hssc,Unknown,Treatable/Rehab,Santa Rosa,County,95407.0,"95407(38.403734, -122.736775)",0
